<img src="./images/DLI_Header.png" style="width: 400px;">

# Assessment

Hopefully, you've learned some valuable skills along the way and had fun doing it. Now it's time to put those skills to the test. In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started! 

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you.

In [2]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights= 'imagenet',
    input_shape=(224, 224, 3),
    include_top= False)

58892288/58889256 [==============================] - 1s 0us/step


## Freeze Base Model

In [3]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

In [5]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))  # Adjust the shape as needed

# Pass inputs through the base model
x = base_model(inputs, training=False)

# Add Global Average Pooling Layer
x = keras.layers.GlobalAveragePooling2D()(x)  # You can also use GlobalMaxPooling2D

# Add final dense layer
num_classes = 6  # Set the number of classes (types of fruit) you have
outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

# Combine inputs and outputs to create the model
model = keras.Model(inputs, outputs)

# Print model summary to check the architecture
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [6]:
model.compile(
    loss='categorical_crossentropy',  # For multi-class classification
    optimizer='adam',  # You can choose other optimizers as well
    metrics=['accuracy']  # Accuracy is a common metric for classification
)

## Augment the Data

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1.0/255,          # Normalize pixel values to [0, 1]
    rotation_range=20,        # Randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,    # Randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,   # Randomly shift images vertically (fraction of total height)
    shear_range=0.2,          # Shear intensity (shear angle in counter-clockwise direction as radians)
    zoom_range=0.2,           # Randomly zoom in/out on images
    horizontal_flip=True,     # Randomly flip images horizontally
    fill_mode='nearest'       # Fill points outside the input boundaries using the nearest available pixel
)

## Load Dataset

In [9]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train',          # Path to the training data directory
                                      target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid',     # Path to the validation data directory
    target_size=(224, 224),
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [10]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs= 25)

Epoch 1/25
37/36 [==============================] - 29s 781ms/step - loss: 1.7450 - accuracy: 0.2479 - val_loss: 1.6042 - val_accuracy: 0.4012
Epoch 2/25
37/36 [==============================] - 24s 645ms/step - loss: 1.5379 - accuracy: 0.4509 - val_loss: 1.4527 - val_accuracy: 0.5866
Epoch 3/25
37/36 [==============================] - 22s 594ms/step - loss: 1.3864 - accuracy: 0.6413 - val_loss: 1.3081 - val_accuracy: 0.6079
Epoch 4/25
37/36 [==============================] - 22s 590ms/step - loss: 1.2701 - accuracy: 0.6438 - val_loss: 1.2185 - val_accuracy: 0.7234
Epoch 5/25
37/36 [==============================] - 22s 585ms/step - loss: 1.1687 - accuracy: 0.7310 - val_loss: 1.1165 - val_accuracy: 0.7842
Epoch 6/25
37/36 [==============================] - 22s 587ms/step - loss: 1.0880 - accuracy: 0.7403 - val_loss: 1.0305 - val_accuracy: 0.7660
Epoch 7/25
37/36 [==============================] - 21s 577ms/step - loss: 1.0203 - accuracy: 0.7707 - val_loss: 0.9852 - val_accuracy: 0.7416

## Unfreeze Model for Fine Tuning

In [11]:
# Unfreeze the base model
base_model.trainable = True  # Set to 'True' to unfreeze the layers

# Compile the model with a low learning rate
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),  # Adjust the learning rate as needed
    loss='categorical_crossentropy',  # For multi-class classification
    metrics=['accuracy']
)

In [12]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs= 10)

Epoch 1/10
37/36 [==============================] - 33s 889ms/step - loss: 0.3584 - accuracy: 0.8782 - val_loss: 0.2816 - val_accuracy: 0.8967
Epoch 2/10
37/36 [==============================] - 24s 660ms/step - loss: 0.2032 - accuracy: 0.9264 - val_loss: 0.1651 - val_accuracy: 0.9331
Epoch 3/10
37/36 [==============================] - 23s 624ms/step - loss: 0.1398 - accuracy: 0.9442 - val_loss: 0.1572 - val_accuracy: 0.9453
Epoch 4/10
37/36 [==============================] - 23s 610ms/step - loss: 0.1079 - accuracy: 0.9560 - val_loss: 0.1935 - val_accuracy: 0.9392
Epoch 5/10
37/36 [==============================] - 22s 599ms/step - loss: 0.0957 - accuracy: 0.9611 - val_loss: 0.1392 - val_accuracy: 0.9483
Epoch 6/10
37/36 [==============================] - 23s 620ms/step - loss: 0.0735 - accuracy: 0.9704 - val_loss: 0.0811 - val_accuracy: 0.9787
Epoch 7/10
37/36 [==============================] - 23s 623ms/step - loss: 0.0645 - accuracy: 0.9788 - val_loss: 0.0762 - val_accuracy: 0.9696

## Evaluate the Model

We now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [13]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

11/10 [================================] - 4s 377ms/step - loss: 0.0713 - accuracy: 0.9787


[0.07129742205142975, 0.978723406791687]

## Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your validation data iterator is called `valid_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [14]:
from run_assessment import run_assessment

In [15]:
run_assessment(model, valid_it)

Evaluating model 5 times to obtain average accuracy...

11/10 [================================] - 4s 382ms/step - loss: 0.0468 - accuracy: 0.9757

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9739.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.
